In [1]:
!nvidia-smi

Tue Apr 28 12:30:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.72       Driver Version: 410.72       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:2D:00.0 Off |                    0 |
| N/A   32C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:31:00.0 Off |                    0 |
| N/A   28C    P0    31W / 250W |   3701MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
%env NVIDIA_VISIBLE_DEVICES 0,1,2,6,7
%env CUDA_VISIBLE_DEVICES 0,1,2,6,7

env: NVIDIA_VISIBLE_DEVICES=0,1,2,6,7
env: CUDA_VISIBLE_DEVICES=0,1,2,6,7


In [4]:
import os
from io import StringIO
from collections import defaultdict
import json

import pandas as pd
from tqdm import tqdm
import IPython

import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

import pickle

In [5]:
cxt = []
with open('data/NCPPolicies_context_20200301.csv', 'r', encoding='utf-8') as f:
    for line in f:
        cxt.append(line.replace('\t', '|!', 1))
cxt = '\n'.join(cxt)
df_cxt_all = pd.read_csv(StringIO(cxt), delimiter='\|!', error_bad_lines=False)
df_cxt = df_cxt_all.set_index('docid')
df_cxt

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,text
docid,
edd1413c78e534afb136f36fdc9c9a00,福建：6部门联合出台暖企措施支持复工稳岗 为解决企业复产的用工困难，经省政府同意，省人社厅、...
230b6fc2a40937f9adf45ea97abad846,化龙桥站至李家坪站区间隧道贯通 12月30日，施工人员在重庆轨道交通九号线一期工程化龙桥站至...
8bab952dfa1a367da1b8e7ad864d766b,吉林多措并举保民生 2月18日，省政府新闻办围绕就业和医疗保障等民生问题召开新闻发布会。省...
ccc5129a465d312b9528b0b04aa4399e,山西加快推进口罩生产供应 新冠肺炎疫情发生后，我省积极督促企业扩产、转产、增产医疗防护物资。...
eb04c2ada51a3161b8477de607721c64,广东：暖企添动力 扩产有信心 2月18日，位于番禺汽车城的广汽新能源，机器人不断挥舞着手臂，...
901e0896408237b398cf13543170fd7b,青海加快信息化建设 支撑“放管服”改革 近日，我省开展了市场监管系统市场主体登记数据质量建设...
a84ad5c0fc7d3755926ec5bf2b2d9dae,重庆出台援企稳岗返还政策 2月16日，记者从市人力社保局获悉，为切实减轻中小企业负担，充分发...
a4c5052b84643fcba77ba4fd2e8da811,云南：各地积极组织企业复工复产 连日来，我省各地在确保疫情防控措施到位的基础上，积极组织当地...
976af8941a4e3e218a31e98fa76a42b8,辽宁智慧助力科技战“疫” 紧急启动两批攻关项目，多项科研工作取得进展 辽宁智慧助力科技战“疫...


In [6]:
df_train = pd.read_csv('data/NCPPolicies_train_20200301.csv', delimiter='\t').drop_duplicates(subset=['question', 'answer']).set_index('id')
df_train

,docid,question,answer
id,,,
47a41a03966431739257ef215cdc1caa,015758c216923f89991ca61c67b29f70,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司
17f3894afaba3f24a3ccd550fe442177,356fb8b804d73c3bab18472c36e67a3f,北京市对于因疫情影响暂停举办的展会项目有何支持政策？,如年内继续在京举办且参展中小微企业数量超过参展企业总数50%的，按照不超过实际缴纳场租费用5...
5f1c9eee4e5a302599f2c5baf674d05a,6488f3b3332b3995a4c60c7e3e3e9993,为切实做好新冠肺炎疫情防控工作，农业农村部启动开展“防控疫情、法治同行”专项法治宣传行动注重...,此次专项法治宣传行动注重以案释法，注重运用新媒体新技术，注重把普法融入到相关立法、执法和管理...
8b85325949b73adc8a91fe7276673ac4,5e996fbd63c83563985bd24222b4a148,浙江省新型肺炎公共服务与管理平台的访问路径是什么？,路径1：打开浙里办APP首页→浙江省新型肺炎公共服务与管理平台 路径2：打开浙里办APP国...
c9e25cd4650d36f488811f65e6ab9a84,99c5f444f76c33589d600d7e8f487067,甘肃省平凉市政务服务中心如何应对疫情问题？,于2月3日（星期一）起，暂停对外办公，具体恢复办理时间待疫情缓解後另行通知。
7721da4299f5337fbcf34eb59cdc43a9,f3bdcc0013733fb7a6a7f37549bb5e62,如何处置哄抬物价、造假等行为？,给予处罚和刑事拘留
380320b7ece133dcae5c2aa6718cba93,34aeec4d64c1367ea48d5c0d64b5054d,工信部向武汉紧急调用了多少物资？,向武汉紧急调用1.4万件防护服、11万双医用手套 自1月23日接到联防联控机制转来武汉物资需...
7dae4c8341543e5b938efe7d9d8c4e5f,902851584a273dab8694f951d2a75718,上海市交通银行只能对受疫情影响较大行业相关企业提供什么支持？,设置直接融资服务绿色通道。
2a1e5bd12d213e83b3ef15c19876d259,2ba59a4e39cf329cbe0d54a97f6b4371,为缓解疫情对企业的影响各地人社部门有何举措？,通过实施援企稳岗、延缴缓缴社会保险费、重点扶持疫情防控企业等举措，出台系列惠企惠民政策，稳定...


In [70]:
df_train[df_train['answer'].str.len() < 20]

,docid,question,answer
id,,,
47a41a03966431739257ef215cdc1caa,015758c216923f89991ca61c67b29f70,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司
7721da4299f5337fbcf34eb59cdc43a9,f3bdcc0013733fb7a6a7f37549bb5e62,如何处置哄抬物价、造假等行为？,给予处罚和刑事拘留
7dae4c8341543e5b938efe7d9d8c4e5f,902851584a273dab8694f951d2a75718,上海市交通银行只能对受疫情影响较大行业相关企业提供什么支持？,设置直接融资服务绿色通道。
66141a935c6733119af885c77a227f6d,4262b3f6696f3694938c678b119a1350,铁岭市对于运输防控重点物资的收入给予什么政策？,免征增值税
8c91f674de77313f98ed410d9eccb4f4,fa00e87358973952ad1ce59a7be6c303,新疆采取什么举措解就业难题？,“线上”举措
d569c06d2710391991852ca5423ad504,4052555c179f3d87b2deca68e7e17a3d,四川省关于免费健康申报证明有效期至什么时候？,至新冠肺炎疫情解除为止
fb5b09d4583239038d85cbe077dff1a3,e52b842ab4493d3db086a5433ec32f26,泰州市为保障弱势群体十个从严期间的基本生活，从什么时间开始，开通爱心服务热线？,2月9日12时
ff1b762cbb34397097aa5c82549092db,7f96e38114723fd49070116527fe47a6,财政部、税务总局公告2020年第9号对企业和个人通过公益性社会组织捐赠用于疫情的现金和物品如...,允许在计算应纳税所得额时全额扣除。
3a2e6100d2a1362094c200ff75906223,488cbdda3a3a30dcb6c43c3bcc1c039c,无法登陆政务服务平台的用人单位和个人如何提交相关资料？,可以通过邮寄、传真等方式


In [68]:
df_train[df_train['question'].str.contains('多少')]

,docid,question,answer
id,,,
380320b7ece133dcae5c2aa6718cba93,34aeec4d64c1367ea48d5c0d64b5054d,工信部向武汉紧急调用了多少物资？,向武汉紧急调用1.4万件防护服、11万双医用手套 自1月23日接到联防联控机制转来武汉物资需...
0251317771f33e809e2e06e7a3ca91c2,fdfae2910346369a94570f0b0ddf5652,长三角铁路累计运送疫情防控物资多少吨？,1294吨。
2dc27b1d3c5436f1bf4121550d6d0260,164d698e7df03dfabde87e732dacf20d,成都推出的减税抗疫贷最多贷多少？,单户最高可贷500万元
f19a7303c3e4382abeb2a59917a9b276,de7aa754c0333de28b94a2efeac463c5,泰州市海陵区审批服务热线是多少？,86512345
c146c825cda03377a4cd01cdd75d8c1a,ceb8f1c546a334e6b7622173420659fb,江西在疫情防控期间对处于失业状态且未达到领取失业保险金条件的参保人员给予多少补助金？,每人每月1000元
c12f171db71e35e08eac2aa3828da030,c430472ef62538a29db9c5e94b249140,山东省各县（市、区）财政按上年补助额度的多少，来提前拨付2019年度病死猪无害化处理补助资金？,50%
535b87f145c13419bfa8c6b97d287016,588cbbd2aa7c3be2bb7f18b92e9832b3,2月16日，全国铁路预计发送旅客多少人次？,85万人
c73aa9cb807a34c6957ea86ffa63ca5e,a684ba0ae6fb326d9a692bce77bc4bd8,重庆两江新区规上工业企业复工比例达到多少？,超过60%。
8e595cc79f623ea1973c4d6aa8a97aad,5e4bee8661ee3c04ad01c67b875caba9,天津港集团连夜抢卸多少吨疫情防控物资？,432吨


In [69]:
df_train[df_train['question'].str.contains('哪里')]

,docid,question,answer
id,,,
c8a7cc5323df3ec0a6332b6419527648,bec4707fcc813202b7e8840a60a1bed4,北京市研发的“京心相助”手机端来京人员信息采集平台，目前在哪里进行试运行？,在朝阳区部分社区
fb64b46df24a372ebe63844eb9a63d1a,585db14147a1382cacdb33d4e7661696,江西省群众在哪里进行线上解决政务问题？,“赣服通”
0f82fa4028c936de84f81eef958fbd6d,2cf8a6e17d3c328ba73cf24e40e7bd2a,上海市群众在哪里上报疫情防控工作问题建议？,登录各区融媒体客户端，或通过各区政府门户网站、政务微信、政务微博等平台
c1dcf557a5873423a10858dc3b1a58af,7db1867d61db319cbdcb5c675603eb49,重庆纳税人在哪里进行线上办理税费？,重庆市电子税务局、自然人电子税务局、“渝快办”、微信公众号、支付宝
9e0ad51bc32431e2bdeaf7a7689f43c3,7a8c2f1b3fd63c37a0cded8346fa22f1,休市期间市民去哪里购买家禽？,可购买经省市农业部门认证的家禽屠宰厂（场）或代宰点屠宰後出厂（场）销售的光禽及冰鲜禽类产品。
dc5fa4d3ffb63d2fa321d10882aa67fe,1c1e54cd2224302cbe205ed24ccb4ffb,乐山市中小企业在哪里查询补贴政策？,《乐山市应对新冠肺炎疫情缓解文旅企业经营困难十条政策措施》《有效应对疫情缓解中小企业生产经营...
fe089b0753f43042987f827739a9b455,f44512aac21e3ff68a49acbc4d6598b1,湖北省黄冈市浠水县正在紧急征用哪里作为应急后备定点收治医院?,县妇幼保健院在建新院区
dacbd7e4fd3a361e945654ba63e9b3e1,b7e72a8455763d618ab0c512aa958606,2月13日中央抗疫小组会议要求把哪里作为疫情防控的重中之重？,武汉市
0c590ebd44953c74b61279858d0c40f9,a195228f5e3531dfaa7beb476062ab8c,“津门战疫”扫码行动的范围包括哪里？,天津市域内机关企事业单位，机场（含候机厅）、车站（含火车、长途客运候车室）、农贸市场、大型超...


In [7]:
eval_set = pd.read_pickle('./tmp/df_val_v7,p')
eval_set

,id,docid,question,answer,text
4549,42ab385fcb093f3aaf516e0a2535a87c,292cbef39ef43c3baa89288c9df27192,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,重大项目复工如何筑牢防控线 战疫情、稳增长、促发展。疫情攻坚战进入关键期，经济领域成为第二战...
393,8e54ffa07e013a4d9eb9bae452ec4306,e74b323c4946398590279d4f474ee738,重庆市纳税人什么时间可以办理发票寄递业务？,2月9日起~2月29日,重庆：20万纳税人受益发票免费寄递 近日，国家税务总局重庆市税务局和中国邮政集团有限...
1164,38150242dcc63085948d031fdb16cb71,1e579ca9544a3cb9ab7d21332f9a8a9d,海东市对近期从武汉等重点地区返回海东人员及其密切接触人员怎么处置？,必须向所在村居、社区登记报告，接受健康管理，并避免到人流密集场所。,海东市新型冠状病毒感染的肺炎疫情处置工作指挥部公告 1月25日24时，青海省启动重大突发公共...
4331,e476613cde7c3f39a9cf0eb30e305aed,e81f12b9f99c3e82ad358c90a52d3e13,天津住房公积金缴存职工可通过哪些途径办理业务？,中心网站、手机APP（天津公积金）、微信公众号（TJZFGJJ）等电子渠道,天津：住房公积金业务实行“网上办” 记者从市住房公积金管理中心获悉，疫情防控期间我市暂停各区...
731,145e3de20a4c3505b38c2981b22db9d2,fb03a2bf6a5f3f95bd7df18e5304219b,上海市公共卫生临床中心如何能做到顶级专家24小时驻扎中心？,打响疫情阻击战第一刻，上海市卫生健康委、申康医院发展中心便统筹安排，抽调全市各大医院具有丰富...,上海重症危重症患者维持“低水平” 各路顶级专家24小时驻扎 上海市公共卫生临床中心自疫情一开...
3119,542e67cb6c3e323d9f7901ba68fb00fa,33ddd81164f63d4cbba1603168a1a2d2,浙江多地为企业推出什么险种保障企业复工？,复工复产企业疫情防控雇主责任险,浙江多地为复工企业定制疫情防控保险产品 “有了这份保单，让我们多了一分安心。”2月22日下午...
3138,9ecb9abec446342cb520fca11d91e8a0,b0f34080da2d381784b37bb5f155dcb9,上海市税收优惠可以便利多少企业？,近20万户,“减免缓退”可为全市企业减负担降成本300亿元以上 上海28条政策打出扶持企业“组合拳” 昨...
1149,ab0b7a071e4e3aaebec8e50bfa484e37,3208a98211b5320dbe979f6d6f4e0add,广东省每年约有多少自然人代开房屋租赁发票？,101万,房屋租赁发票可以网上代开 广东“非接触”式办税新举措正在珠海、中山等五地试点 此次税务部门在...
151,fb64b46df24a372ebe63844eb9a63d1a,585db14147a1382cacdb33d4e7661696,江西省群众在哪里进行线上解决政务问题？,“赣服通”,江西省人民政府办公厅关于印发积极应对当前疫情充分依托“赣服通”平台为企业和群众提供优质高效服...
472,03cc3eb9c6e033bd9e7db38ca4fa93c0,f5c1ed72c7bd375a8efc362d65833f99,山东通过哪些方式来优化网上办税缴费平台？,加强电子税务局、手机APP等办税缴费平台的运行维护和应用管理，确保系统安全稳定。优化电子税务...,国家税务总局关于充分发挥税收职能作用 助力打赢疫情防控阻击战若干措施的通知 国家税务总局...


In [8]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch(['10.224.54.36'])

import pickle
addr = pickle.load(open('./addr_dict.p', 'rb'))

locs = []

for prov in addr['provinces']:
    dprov = addr[prov]
    locs.append(prov if len(dprov['alias']) == 0 else dprov['alias'][0])
    
    for city in dprov['cities']:
        dcity = dprov[city]
        locs.append(city if len(dcity['alias']) == 0 else dcity['alias'][0])
        
import re

loc_pattern = re.compile('|'.join(locs))

def get_locs(s):
    return list((set([f.group() for f in re.finditer(loc_pattern, s)])))

def search_with_loc(question, n):
    doc = {
                "size": n, 
                "query": {
                    "bool": {
                        "must": [
                            {
                                "match": {
                                    "text": {
                                        "query": question,
                                        "minimum_should_match": f"50%"
                                    }
                                },
                            }
                        ],
                                
                        "should": [
                            {
                                "match": {
                                    "locs": ' '.join(get_locs(question))
                                }
                            }
                        ]
                    }
                }
            }
    try:
        _searched = es.search(index='yqzw', doc_type='_doc', body=doc)
        answers = []
        for item in _searched['hits']['hits']:
            answers.append({
                'docid': item['_source']['docid'],
                'text': item['_source']['text'],
                'locs': item['_source']['locs'],
                'score': item['_score']
            }) 
        return answers

    except:
        traceback.print_exc()

In [9]:
pd.DataFrame(search_with_loc('北京市项目复工后应采取什么样的消毒方法', 10))

,docid,locs,score,text
0,a7bde7d661fb397ea6a081840f936950,"[武汉, 湖北, 北京]",27.777836,北京市住房和城乡建设委员会关于进一步加强疫情防控保障建设工程安全有序复工的通知 各区住房城乡...
1,5e4f22ca7d09374c8029f0ae09c08ffa,"[上海, 北京]",26.527319,营商环境“优”无止境 如果说基础设施是企业发展的硬环境，那么政府的服务就是企业发展的软环境。...
2,eded6725f37a35eaaab09524f4eff5e4,"[武汉, 湖北, 北京, 荆门, 吉林]",26.279804,10家中央企业：抗疫物资不涨价！ 新华社北京2月1日电 题：10家中央企业：抗疫物资不涨价！...
3,4b19b768c42f35368eb22fe249e7c4b9,[北京],25.015512,北京市住房和城乡建设委员会关于印发《施工现场新型冠状病毒突发疫情应急预案》的通知 各区住房城...
4,8658892dedb534ce8dd36b2bdf1820a6,"[湖北, 北京]",24.820263,北京市住房和城乡建设委员会关于进一步加强新冠肺炎疫情防控期间施工现场管理的通知 各区住房城乡...
5,292cbef39ef43c3baa89288c9df27192,[北京],23.760970,重大项目复工如何筑牢防控线 战疫情、稳增长、促发展。疫情攻坚战进入关键期，经济领域成为第二战...
6,a4a8819e2060320b9f542a7755e394c2,"[湖北, 孝感, 重庆]",23.091915,重庆 重庆日报2020-02-20 07:51:31 从1月26日起，重庆市陆续派出3批次3...
7,2067a5d9cfed32ccb86e8e0bfba6cea9,"[湖北, 北京]",23.022007,北京市住房和城乡建设委员会关于施工现场新型冠状病毒感染的肺炎疫情防控工作的通知 各区住房城乡...
8,6903a41b10e03bac9c0af82645945e04,"[山东, 河北, 朝阳, 浙江, 广州, 菏泽, 北京]",22.775972,雪中战“疫”：探访风雪中复工的北京建筑工地 新华社北京2月14日电 题：雪中战“疫”：探访风...
9,e516f2047498368792267cd5afd7aa2d,[北京],22.725956,北京市住房和城乡建设委员会关于印发《施工现场新型冠状病毒感染的肺炎疫情防控工作管理规定》的通...


In [12]:
tmp = []

for row in tqdm(eval_set.itertuples(), total=len(eval_set)):
    _df = pd.DataFrame(search_with_loc(row.question, 30))
    _df['question'] = row.question
    _df['answer'] = row.answer
    _df['id'] = row.id
    tmp.append(_df)
    
df_eval = pd.concat(tmp, axis=0)
df_eval

100%|██████████| 497/497 [00:05<00:00, 94.35it/s]


,docid,locs,score,text,question,answer,id
0,a7bde7d661fb397ea6a081840f936950,"[武汉, 湖北, 北京]",27.777836,北京市住房和城乡建设委员会关于进一步加强疫情防控保障建设工程安全有序复工的通知 各区住房城乡...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
1,5e4f22ca7d09374c8029f0ae09c08ffa,"[上海, 北京]",26.527319,营商环境“优”无止境 如果说基础设施是企业发展的硬环境，那么政府的服务就是企业发展的软环境。...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
2,eded6725f37a35eaaab09524f4eff5e4,"[武汉, 湖北, 北京, 荆门, 吉林]",26.279804,10家中央企业：抗疫物资不涨价！ 新华社北京2月1日电 题：10家中央企业：抗疫物资不涨价！...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
3,4b19b768c42f35368eb22fe249e7c4b9,[北京],25.015512,北京市住房和城乡建设委员会关于印发《施工现场新型冠状病毒突发疫情应急预案》的通知 各区住房城...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
4,8658892dedb534ce8dd36b2bdf1820a6,"[湖北, 北京]",24.820263,北京市住房和城乡建设委员会关于进一步加强新冠肺炎疫情防控期间施工现场管理的通知 各区住房城乡...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
5,292cbef39ef43c3baa89288c9df27192,[北京],23.760970,重大项目复工如何筑牢防控线 战疫情、稳增长、促发展。疫情攻坚战进入关键期，经济领域成为第二战...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
6,a4a8819e2060320b9f542a7755e394c2,"[湖北, 孝感, 重庆]",23.091915,重庆 重庆日报2020-02-20 07:51:31 从1月26日起，重庆市陆续派出3批次3...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
7,2067a5d9cfed32ccb86e8e0bfba6cea9,"[湖北, 北京]",23.022007,北京市住房和城乡建设委员会关于施工现场新型冠状病毒感染的肺炎疫情防控工作的通知 各区住房城乡...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
8,6903a41b10e03bac9c0af82645945e04,"[山东, 河北, 朝阳, 浙江, 广州, 菏泽, 北京]",22.775972,雪中战“疫”：探访风雪中复工的北京建筑工地 新华社北京2月14日电 题：雪中战“疫”：探访风...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c
9,e516f2047498368792267cd5afd7aa2d,[北京],22.725956,北京市住房和城乡建设委员会关于印发《施工现场新型冠状病毒感染的肺炎疫情防控工作管理规定》的通...,北京市项目复工后应采取什么样的消毒方法？,1.室内空气 开窗通风为主，每日开窗通风2-3次，每次至少30分钟，注意人员保暖。 ...,42ab385fcb093f3aaf516e0a2535a87c


In [11]:
from transformers import AutoModelForQuestionAnswering, BertTokenizer, AutoConfig
from question_answering_pipeline import QuestionAnsweringPipeline
import torch
import re

In [13]:
model = AutoModelForQuestionAnswering.from_pretrained('./models/albert-chinese-large-v8/checkpoint-7000/')
tokenizer = BertTokenizer.from_pretrained('./models/albert-chinese-large-v8/checkpoint-7000/')

pip = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer, device=6)

In [14]:
eval_preds = pip(
    question=df_eval.question.tolist(),
    context=df_eval['text'].str.replace(r'\s+', ' ').str.strip().str.slice(stop=5000).tolist(), 
    max_answer_len=250, max_seq_len=512, max_question_len=60, doc_stride=128, topk=20
)

100%|██████████| 14757/14757 [23:28<00:00,  4.97it/s]


In [15]:
pickle.dump(eval_preds, open('tmp/val-preds-v8-30.p', 'wb'))

In [17]:
eval_preds[0]

[{'score': 0.9522050446963277,
  'null_score': 0.9522050446963277,
  'start': 0,
  'end': 0,
  'answer': ''},
 {'score': 0.00016040411274931277,
  'null_score': 0.9522050446963277,
  'start': 3872,
  'end': 3991,
  'answer': '每日对办公室、宿舍、门岗、会议室、卫生间、淋浴房等重点区域进行不少于两次的预防性消毒，消毒方法请参照市疾病预防控制中心发布的《新型冠状病毒感染的肺炎流行期间预防性消毒指引》执行。同时要保持室内环境清洁，每天通风2-3次，每次至少30分钟。'},
 {'score': 6.797356503863636e-05,
  'null_score': 0.9522050446963277,
  'start': 3874,
  'end': 3991,
  'answer': '对办公室、宿舍、门岗、会议室、卫生间、淋浴房等重点区域进行不少于两次的预防性消毒，消毒方法请参照市疾病预防控制中心发布的《新型冠状病毒感染的肺炎流行期间预防性消毒指引》执行。同时要保持室内环境清洁，每天通风2-3次，每次至少30分钟。'},
 {'score': 5.322382733272818e-05,
  'null_score': 0.9522050446963277,
  'start': 3872,
  'end': 3991,
  'answer': '每日对办公室、宿舍、门岗、会议室、卫生间、淋浴房等重点区域进行不少于两次的预防性消毒，消毒方法请参照市疾病预防控制中心发布的《新型冠状病毒感染的肺炎流行期间预防性消毒指引》执行。同时要保持室内环境清洁，每天通风2-3次，每次至少30分钟。'},
 {'score': 2.860354542409682e-05,
  'null_score': 0.9522050446963277,
  'start': 3872,
  'end': 3914,
  'answer': '每日对办公室、宿舍、门岗、会议室、卫生间、淋浴房等重点区域进行不少于两次的预防性消毒，'},
 {'score': 2.52

In [248]:
from rouge.rouge import Rouge
from difflib import SequenceMatcher
import copy
import re

import thulac
thu = thulac.thulac(seg_only=True, user_dict='user_dict.txt')

import sys
sys.setrecursionlimit(10000)

rouge = Rouge()

def spacify(s):
    return re.sub(r'\s+', ' ', ' '.join(list(s)))

def get_result(model_preds, df_search, a, b, c, w1, w2, w3, w4):
    model_preds = copy.deepcopy(model_preds)
    
    max_answer_len = 2500
    answers = []
    for i, p in enumerate(model_preds):
        cands = []
        min_null_score = 10000
        for j, d in enumerate(p):
            if d['answer'] == '':
                min_null_score = min(min_null_score, d['score'])
            else:
                d['score1'] = c * d['score'] + (1 - c) * (1 - d['null_score'])
            
                
#                 if len(d['answer']) < 2 or len(d['answer']) > max_answer_len:
#                     d['score1'] = max(0, d['score1'] - 0.1)
                cands.append(d)

        cands = sorted(cands, key=lambda x:x['score1'], reverse=True)
        
        s1 = cands[0]['score1'] if len(cands) > 0 else 0
        s2 = cands[1]['score1'] if len(cands) > 1 else 0
        s3 = cands[2]['score1'] if len(cands) > 2 else 0
        s4 = cands[3]['score1'] if len(cands) > 3 else 0
        
        score_context = w1 * s1 + w2 * s2 + w3 * s3 + w4 * s4

        best_answer = cands[0]['answer'].strip()

        if min_null_score > 1:
            min_null_score = 0
            
        if len(best_answer) < 3 or len(best_answer) > max_answer_len:
            score_context = score_context - 0.5
            
        answers.append((best_answer, min_null_score, score_context))
        
    df_preds = pd.DataFrame(answers, columns=['pred', 'null_prob', 'prob'])

    df_search_pred = pd.concat([df_search.reset_index(drop=True), df_preds], axis=1)
#         df_search_pred = df_search_pred[df_preds['null_prob'] < thres]

    df_search_pred['score_all'] = df_search_pred['score'] * (1 - a - b) / 100 + df_search_pred['prob'] * a + (1 - df_search_pred['null_prob']) * b

    df_pred_select = []
    for k, _df in df_search_pred.groupby('id'):
        df_pred_select.append(_df.sort_values('score_all', ascending=False).iloc[0])

    df_pred_select = pd.DataFrame(df_pred_select)

    answer_pred_refine = []
    for row in df_pred_select.itertuples():
        matches = [m for m in SequenceMatcher(a=row.text, b=row.pred).get_matching_blocks() if m.size > 0]
        refine = row.text[matches[0].a: matches[-1].a + matches[-1].size]
        
        old_refine = copy.copy(refine)
        
        refine = refine.strip('，').strip(':').strip('：').strip('（').strip('；').strip('、').strip()
        
        if re.sub(r'\s+', '', refine) != re.sub(r'\s+', '', row.pred):
            refine = row.pred
        
        if len(refine) > 5 and len(re.findall('[。，：]', refine[:4])) > 0:
            for f in re.finditer('[。，：、]', refine[:4]):
                refine = refine[f.span()[0] + 1:]
                break
                
            print('----截取开头标点----')
        
        
        if re.match(r'^[的].*', refine):
            refine = refine[1:]
            print('----去掉开头停用词----')
            
        if matches[0].a != 0:
            tmp = row.text[matches[0].a - 1: matches[-1].a + matches[-1].size]
            tw = [w[0] for w in thu.fast_cut(tmp)]
            rw = [w[0] for w in thu.fast_cut(refine)]
            
            if len(rw) > 0 and rw[0] != tw[0] and rw[0] in tw[0]: # 少了一个字
                print('----去掉开头不完整词语----')
                refine = tmp[len(tw[0]):]
                
#         if matches[-1].a + matches[-1].size < len(row.text):
#             tmp = row.text[matches[0].a: matches[-1].a + matches[-1].size + 1]
#             tw = [w[0] for w in thu.fast_cut(tmp)]
#             rw = [w[0] for w in thu.fast_cut(refine)]
#             if len(rw) > 0 and rw[-1] != tw[-1] and rw[-1] in tw[-1]:
#                 print('----去掉结尾不完整词语----')
#                 refine = tmp[: len(tmp) - len(tw[-1])]
                
        refine = refine.strip('，').strip(':').strip('：').strip('（').strip('；').strip('、').strip()
                
        if old_refine != refine:
            print('old:', old_refine)
            print('new:', refine)
            
        answer_pred_refine.append(refine)

    df_pred_select['answer_pred_refine'] = answer_pred_refine

    return df_pred_select

Model loaded succeed


In [247]:
thu.fast_cut('重要生活物')

[['重要', ''], ['生活物', '']]

In [249]:
df_pred_select = get_result(eval_preds, df_eval, 0.3, 0, 0, 1, 0, 0, 0)
rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']

----去掉开头不完整词语----
old: 次、1294吨。
new: 1294吨。
old: 税务部门将为纳税人提供预约服务，错期错峰安排办理，
new: 税务部门将为纳税人提供预约服务，错期错峰安排办理
----截取开头标点----
old: 说，2020年2月 17日 00:00 至免通终止时间内，ETC车辆计费为 0 元。
new: 2020年2月 17日 00:00 至免通终止时间内，ETC车辆计费为 0 元。
----去掉开头不完整词语----
old: 过24小时电话热线33672885、33682885
new: 24小时电话热线33672885、33682885
old: 实行1个月房租免收、2个月租金减半优惠政策，
new: 实行1个月房租免收、2个月租金减半优惠政策
old: 成都市税务局与建设银行四川省分行联合推出小微企业“减税抗疫贷”，对防疫物资生产企业加大优惠利率信贷支持力度，
new: 成都市税务局与建设银行四川省分行联合推出小微企业“减税抗疫贷”，对防疫物资生产企业加大优惠利率信贷支持力度
----去掉开头不完整词语----
old: 予75%
new: 75%
old: 约65万个，
new: 约65万个
old: 着力稳定粮食生产，加快恢复生猪生产，积极抓好蔬菜生产，统筹抓好家禽、草食牲畜生产以及抓好动植物疫情防控等工作，做到当前生产和长远谋划两不误。（
new: 着力稳定粮食生产，加快恢复生猪生产，积极抓好蔬菜生产，统筹抓好家禽、草食牲畜生产以及抓好动植物疫情防控等工作，做到当前生产和长远谋划两不误。
----去掉开头不完整词语----
old: 确185个
new: 185个
----截取开头标点----
old: 绍，“减税抗疫贷”是成都市税务局与建行四川省分行为帮助受疫情影响的小微企业渡过难关联合推出的一款小微信贷产品。作为税银合作“减税云贷”系列产品，“减税抗疫贷”以纳税企业的实缴税额及减免税额为授信依据，重点面向诚信纳税及享受税收减免且生产销售疫情防控重点保障物资的小微企业发放。纯信用，无需任何抵押或担保，单户最高可贷500万元，重点防疫企业还可享受专门利率优惠。
new: “减税抗疫贷”是成都市税务局与建行四川省分行为帮助受疫情影响的小微企业渡过难关联合推出的一款小微信贷产品。作为税银合作“减税云贷”系列产品，“减

----去掉开头不完整词语----
old: 施28个，
new: 28个
----去掉开头停用词----
old: 的《关于进一步明确疫情防控期间返京人员有关要求的通告》
new: 《关于进一步明确疫情防控期间返京人员有关要求的通告》
----去掉开头不完整词语----
old: 户3个月综合服务费约200万元，
new: 3个月综合服务费约200万元
old: 对疫情防控物资生产企业生产经营所需的水、电、气等，
new: 对疫情防控物资生产企业生产经营所需的水、电、气等
----去掉开头不完整词语----
old: 低0.5个百分点
new: 0.5个百分点
----去掉开头不完整词语----
old: 台《云南省建筑施工疫情防控措施》，
new: 《云南省建筑施工疫情防控措施》
old: 从13个扩增到30个，
new: 从13个扩增到30个
----去掉开头不完整词语----
old: 忌“盲目封村”“一封了之”“一堵了之”，切忌形式主义、官僚主义，防止一刀切。
new: “盲目封村”“一封了之”“一堵了之”，切忌形式主义、官僚主义，防止一刀切。
----去掉开头不完整词语----
old: 打12366纳税服务热线进行电话咨询，也可以通过重庆税务微信公众号、官方微博进行网上咨询。
new: 12366纳税服务热线进行电话咨询，也可以通过重庆税务微信公众号、官方微博进行网上咨询。
----截取开头标点----
old: 员，一律由乡镇（办事处）集中隔离观察，费用自理。
new: 一律由乡镇（办事处）集中隔离观察，费用自理。
----去掉开头不完整词语----
old: 过12366纳税服务热线、微信等渠道
new: 12366纳税服务热线、微信等渠道
old: 充分利用公示栏、宣传栏、小广播、业主微信群、电梯间广告等，
new: 充分利用公示栏、宣传栏、小广播、业主微信群、电梯间广告等
----去掉开头不完整词语----
old: 端、12366
new: 12366
old: 通过电话、短信、微信等方式，
new: 通过电话、短信、微信等方式
old: 超过160亿元，
new: 超过160亿元
----去掉开头不完整词语----
old: 不得提前开学，同时有效告知学生不能提前返校，具体开学时间根据疫情防控形势，按照市委市政府及上一级主管部门的统一要求决

{'f': 0.7553333180398645, 'p': 0.7904925307198558, 'r': 0.7856272524430629}

In [243]:
%debug

> <ipython-input-241-a4db02b333df>(105)get_result()
    103             tw = [w[0] for w in thu.fast_cut(tmp)]
    104             rw = [w[0] for w in thu.fast_cut(refine)]
--> 105             if len(rw) > 0 and rw[-1] != tw[-1] and rw[-1] in tw[-1]:
    106                 print('----去掉结尾不完整词语----')
    107                 refine = tmp[: len(tmp) - len(tw[-1])]

ipdb> rw
['北京', '小汤山', '医院']
ipdb> rw[-1]
'医院'
ipdb> tw
[]
ipdb> q


In [ ]:
# 0.707 ./models/albert-chinese-large-v8/checkpoint-7000/ 10 0.7 0 0.2 
# 0.741 ./models/albert-chinese-large-v8/checkpoint-7000/ 30 0.3 0 0 1 0 0 0
# 0.7553333180398645 ./models/albert-chinese-large-v8/checkpoint-7000/ 30 0.3 0 0 1 0 0 0 nlp

In [93]:
for s in df_pred_select['answer_pred_refine'].str.slice(-2, -1).value_counts().reset_index().itertuples():
    if not is_chinese(s.index):
        print(s)

Pandas(Index=0, index='。', answer_pred_refine=19)
Pandas(Index=2, index='0', answer_pred_refine=10)
Pandas(Index=12, index='2', answer_pred_refine=6)
Pandas(Index=15, index='4', answer_pred_refine=5)
Pandas(Index=16, index='7', answer_pred_refine=5)
Pandas(Index=20, index='5', answer_pred_refine=5)
Pandas(Index=21, index='1', answer_pred_refine=5)
Pandas(Index=22, index='', answer_pred_refine=4)
Pandas(Index=24, index='3', answer_pred_refine=4)
Pandas(Index=26, index='9', answer_pred_refine=4)
Pandas(Index=31, index='6', answer_pred_refine=4)
Pandas(Index=32, index='”', answer_pred_refine=4)
Pandas(Index=35, index='n', answer_pred_refine=3)
Pandas(Index=38, index='P', answer_pred_refine=3)
Pandas(Index=41, index='）', answer_pred_refine=3)
Pandas(Index=77, index='8', answer_pred_refine=2)
Pandas(Index=91, index='%', answer_pred_refine=2)
Pandas(Index=103, index='/', answer_pred_refine=1)
Pandas(Index=126, index='…', answer_pred_refine=1)
Pandas(Index=141, index='’', answer_pred_refine=1

In [143]:
df_pred_select[df_pred_select['answer_pred_refine'].str.len() < 4]

,docid,locs,score,text,question,answer,id,pred,null_prob,prob,score_all,answer_pred_refine
5488,9ec52ce4ac8738ae856f7454e9ad3fa0,"[武汉, 湖北, 山东, 河南, 云南, 青岛, 黑龙江, 哈尔滨, 辽宁, 江西, 郑州,...",35.111900,“菜篮子”里的力量——全国各地努力做好保障和改善民生工作在行动 新华社北京2月18日电 题：...,黑龙江从几个方面出台就业扶持政策？,7个,11f543b996f735978fc84425263a67f2,从7个,0.000000,0.999253,0.545559,从7个
2151,7b5f4e800f9f3bac98e23cb535dfb2d5,"[黑河, 哈尔滨, 黑龙江]",88.381325,黑龙江：“指尖”备春耕 生产抗疫两不误 新华社哈尔滨2月16日电（记者 王建）在手机屏幕上，...,黑龙江省供销社推出了什么形式的订购农资服务？,网上订购农资服务,4ed049bc609633928776ed8cb902de1d,网上,0.000000,0.798643,0.858262,网上
8334,088eaa7dfd76353e9ffa9bc3c40ed704,[漯河],113.710370,漯河市市长刘尚进主持召开市政府 2月20日上午，漯河市市长刘尚进带队走进企业和药店，专题调研...,2020年新冠肺炎疫情发生以来，河南省漯河市连续几次召开服务企业周例会，研究医疗防护物资、市...,4次,51e56c2643f8335fa3c8cc6c4baf4e55,续4次,0.001445,0.998555,1.095539,续4次
14737,8aa2810b103c3a21a395bbb34132a9c4,"[浙江, 丽水]",40.818190,市信访局关于做好近期群众来访接待工作的通告 为深入贯彻中央、省、市关于新型冠状病毒感染的肺炎...,丽水市民信访APP叫什么？,浙里办,57afcca081603e7e9f0fb17d5c613425,浙里办,0.931074,0.068926,0.306405,浙里办
5656,1960ba2daed333a6b1bf96cac009f638,[],73.941560,横琴出台“暖企10条”助力企业复工复产 据悉，在商事登记、商标注册方面，横琴新区工商局为横琴...,横琴新区工商局的商标注册寄送服务如何收费？,免费,6ab2d0d27ffa3b5faaff64de074ddb20,免费,0.000000,0.799987,0.757587,免费
10517,c88fa49719a13d0cba585fd6e298aaaf,[],48.628677,助力小微企业渡难关 微众银行推出APP办业务免结算手续费 作者：齐金钊 2020-02-25...,微众银行“不还本只还息”政策延期了多长时间？,三个月,8025ab809aa23cee94331c2258a547b7,三个月,0.010882,0.989118,0.637136,三个月
8933,67ce546d4ca2301ba572b9cb72621c4c,[北京],61.552110,国务院办公厅转发国家卫生健康委、人力资源社会保障部、财政部《关于改善一线医务人员工作条件切实...,国家卫生健康委为落实改善一线医务人员工作条件提出几个方面完善措施？,七个方面,84660040f24b31739b15078575adeecd,七方面,0.023277,0.976723,0.723882,七方面
738,989ceda97a5130bda0508200b385dbd8,"[武汉, 湖北, 北京]",42.679610,新闻办就湖北组织开展疫情防控和医疗救治工作情况举行发布会 国新办就湖北组织开展疫情防控和医疗...,湖北武汉开放了几个方舱医院？,9个,8da1181652313d6b97fa6de829e94334,了9个,0.000000,0.999245,0.598531,了9个
6616,5cab9cef664e33bcbf2e9c649330d2bd,"[浙江, 温州]",50.604940,温州将按“三个时段”推进复工复产 新华社温州2月12日电（记者 王俊禄）记者12日从浙江省温...,温州面上的企业复工复产分几个时间段?,三个,b1911d3e72f63231be2b94705688429d,三个,0.000000,0.795501,0.592885,三个
708,52933faef4873b679e473ae970826633,[],63.489033,农业农村部开通“春耕农机线上服务站” 眼下，全国即将进入繁忙的农机春耕备耕阶段。日前，农业农...,农业农村部开通的“春耕农机线上服务站”有几方面功能？,四方面,b96efa0e421c3912a98e2d8afa1dfae6,四方面,0.006688,0.993312,0.742417,四方面


In [92]:
from squad import is_chinese

In [129]:
df_pred_select[df_pred_select['answer_pred_refine'].str.match(r'^.*:$')]

,docid,locs,score,text,question,answer,id,pred,null_prob,prob,score_all,answer_pred_refine


In [125]:
_['answer_pred_refine'].iloc[0]

'推荐成功上岗人数为50人（含）以上的，补助2.4万元；100人（含）以上的，补助4.5万元；200人（含）以上的，补助9万元。一'

In [76]:
df_pred_select[(df_pred_select['answer'].str.len() < 20) & （]

,docid,locs,score,text,question,answer,id,pred,null_prob,prob,score_all,answer_pred_refine
1821,49dff71e8a8231e89915b1e08e617639,[],33.961582,市发改委组织党员进社区开展疫情防控工作 为进一步深入贯彻习近平总书记关于疫情防控工作的重要讲...,党员进社区开展疫情防控工作都去了哪些社区？,西苑、洛耐社区,01040d05af9d3ef7baf7cd8c13fd1746,西苑、洛耐社区,0.000000,0.922465,0.514470,西苑、洛耐社区
12718,cd6c71504c583325bee2181ab4da36d0,[],72.556725,民航局：学生旅客3月31日前机票可免费退改签 为进一步做好新型冠状病毒肺炎防控工作，根据各地...,哪些旅客3月31日前机票可免费退改签？,学生旅客,0160d7f5276331a682d16e7fe73fc45e,学生旅客3,0.000000,0.953490,0.793944,学生旅客3
12928,fdfae2910346369a94570f0b0ddf5652,"[杭州, 阜阳, 南京, 温州, 台州, 浙江, 合肥, 徐州, 上海, 苏州]",85.711580,长三角铁路“一日一图”适时增减列车 车上预留隔离座位 新华社上海2月9日电（记者 贾远琨）随...,长三角铁路累计运送疫情防控物资多少吨？,1294吨。,0251317771f33e809e2e06e7a3ca91c2,次、1294吨。,0.000000,0.997065,0.899101,次、1294吨。
14257,97b1b1956e1a3af391762aa69654a754,"[武汉, 广州]",54.971070,广州市物业行业疫情防控工作情况 市住房城乡建设局认真贯彻落实中央、省、市精神，根据疫情特点，...,广州市务工人员抵穗当天如何申报个人相关信息？,通过“穗康”微信小程序,02e634046fa0348ba98272644355f802,通过“穗康”微信小程序,0.000000,0.999540,0.684659,通过“穗康”微信小程序
3556,19c96505ce6538d6ae97254bd2a29518,[重庆],29.566977,@重庆人：这些税收优惠政策请收藏！ 近期，从国家到地方，出台了多项支持企业和个人抗击疫情、复...,税务局对取得单位发放的医药如何征税？,免征个人所得税。,035f627c19103f65b15d3e61e35e0bd0,免征个人所得税；对因疫情影响导致职工生活困难，单位发放给职工的生活困难补助等福利费，免征个人...,0.000000,0.999868,0.506929,免征个人所得税；对因疫情影响导致职工生活困难，单位发放给职工的生活困难补助等福利费，免征个人...
12599,20f3c1dd8e22399e8f4c3844d40461cd,"[惠州, 广东, 江门, 肇庆]",51.915585,广东出台2.0版“促进就业九条” 多举措支持企业稳岗 三大就业工程带动重点群体就业 2月20...,广东省对2019年降低保险费率政策的实施期限做了怎么样的调整？,延长至2021年4月30日。,061a217d3f11362fa2df222a2b6a174b,延长至2021年4月30日。,0.034067,0.965933,0.653189,延长至2021年4月30日。
9798,fb75f41dcab7341aabb4ce6056b1afc5,[湖北],96.076180,湖北省新型冠状病毒感染肺炎疫情防控指挥部通告 为切实做好新冠肺炎疫情防控期间滞留在鄂外地人员...,湖北省新型冠状病毒感染肺炎疫情防控指挥部为鄂外地人有生活困难的人员给予何种保障？,由各地设置集中安置点，提供食宿、医疗等,0668a88077ab3564b8969fe03793a319,由各地设置集中安置点，提供食宿、医疗等基本生活保障。,0.000000,0.988340,0.969035,由各地设置集中安置点，提供食宿、医疗等基本生活保障。
8424,d1dfb56ab7733109b28302154a04ff0b,[内江],76.116240,内江市应对新型冠状病毒感染肺炎疫情应急指挥部通告 （第14号） 为畅通献爱心渠道，整合社会资...,内江市哪些慈善组织取得了公开募捐资格？,内江市红十字会和各县（市、区）红十字会,0b07591fdf9d364d81ae636a11bc467e,内江市红十字会和各县（市、区）红十字会,0.000276,0.999724,0.832731,内江市红十字会和各县（市、区）红十字会
3226,73ec5586ab033968b3b47f9ac326d4bf,[芜湖],40.709335,关于做好疫情防控期间我市企业 复工工作的通知 各县（市）、区人民政府，省江北产业集中区、经济...,芜湖对疫情重点地区及较重地区企业员工有何要求？,不得返芜。,0be3e9ce1a383034aec61108be6e42fe,不得返芜。,0.416536,0.583464,0.460005,不得返芜。
3766,7b3cc45b186a3266966f001e6bd8a25d,[北京],38.006516,北京小汤山医院正在改造 将作为抗击疫情的后备补充 新华社北京1月30日电（记者 李嘉瑞、侠克...,北京市小汤山康复医院的前身是哪家医院？,北京小汤山医院,0d7b7c0059fb3faf98714fa039e96c05,北京小汤山医院,0.000000,0.944846,0.549499,北京小汤山医院


In [66]:
for row in df_pred_select.itertuples():
    if re.sub(r'\s+', '', row.answer_pred_refine) != re.sub(r'\s+', '', row.pred):
        print(row.question)
        print(row.pred)
        print(re.sub(r'\s+', ' ', row.answer_pred_refine))
        print(row.answer)

In [233]:
for row, r1, r2 in zip(df_pred_select.itertuples(),
                       rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify)),
                       rouge.get_scores(df_pred_select['pred'].apply(spacify), df_pred_select['answer'].apply(spacify))):
    if r2['rouge-l']['f'] > r1['rouge-l']['f']:
        print(row.pred)
        print(row.answer_pred_refine)
        print(row.answer)

纳税期限统一为“季度”，
纳税期限统一为“季度”
重庆市税务局还将个体户的纳税期限统一为“季度”，进一步减少纳税人办税次数。
去商超取菜，要量体温，体温正常才能进门；
去商超取菜，要量体温，体温正常才能进门
外来返沪经营人员严格居家隔离；所有工作人员、从业人员和市民进入菜场商超必须测量体温，未测体温不得进场；测量体温情况必须有记录；所有进入菜场商超人员必须规范佩戴口罩，不戴口罩不得进入；每天必须做好开休市消毒至少2次；必须有疫情防控宣传和告示等。
统http://sbj.cnipa.gov.cn/wssq/
http://sbj.cnipa.gov.cn/wssq/
申请人自行通过网上服务系统在线提交商标申请的，选择“网上申请用户登录”。已注册网申用户的，可直接登录办理业务；未注册的，先注册后再提交申请
影剧院、棋牌室、游艺厅、网吧、KTV、公共浴室、足浴店和小餐饮店开业。
棋牌室、游艺厅、网吧、KTV、公共浴室、足浴店和小餐饮店开业。
暂时禁止影剧院、棋牌室、游艺厅、网吧、KTV、公共浴室、足浴店和小餐饮店开业。
将药品、医疗器械注册收费标准降低至零元，
将药品、医疗器械注册收费标准降低至零元
在帮扶企业方面，科学有序地帮助生产企业复工复产，延长合同履行期限，支持企业扩能转产，缓缴社会保险费及医疗保险，适度减免租金，延期缴纳水电气费，完善企业信用修复机制。
至2020年12月31日。
2020年12月31日。
截至2020年12月31日。
享受无偿献血者荣誉和待遇，
享受无偿献血者荣誉和待遇
享受无偿献血者荣誉和待遇，捐献血浆200毫升视为捐献一次全血。献血浆后，发放《献血证》并将献血浆者信息录入全国血液管理信息系统，按规定参与无偿献血表彰活动。
同机、同厢、同车的人员信息，全部登记造册，并向当地卫生健康部门提供；
同厢、同车的人员信息，全部登记造册，并向当地卫生健康部门提供
同机、同厢、同车的人员信息，全部登记造册，并向当地卫生健康部门提供
鼓励国有企业减免商贸服务企业生产经营用房租金、有序引导农民工返岗、加大信贷支持等措施，
鼓励国有企业减免商贸服务企业生产经营用房租金、有序引导农民工返岗、加大信贷支持等措施
鼓励国有企业减免商贸服务企业生产经营用房租金、有序引导农民工返岗、加大信贷支持等措施，积极支持企业复工复产。
的《关于进一步加强本市公共汽电车、出租汽车

In [240]:
a = 0
b = 0
c = 0
w1 = 1
w2 = 0
w3 = 0
w4 = 0

for b in np.arange(0, 1.01, 0.1):
    if b > 1 - a:
        continue

    if w2 > w1 or w3 > w2 or w4 > w3:
        continue

    df_pred_select = get_result(eval_preds, df_eval, a, b, c, w1, w2, w3, w4)

    rst.append((a, b, c, w1, w2, w3, w4, rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']))
    print(rst[-1])
    
sorted(rst, key=lambda x:x[-1]['f'], reverse=True)

----去掉开头不完整词语----
old: 次、1294吨。
new: 1294吨。
old: 山东省电子税务局、手机APP、邮寄等“非接触式”途径，
new: 山东省电子税务局、手机APP、邮寄等“非接触式”途径
----截取开头标点----
old: 说，2020年2月 17日 00:00 至免通终止时间内，ETC车辆计费为 0 元。
new: 2020年2月 17日 00:00 至免通终止时间内，ETC车辆计费为 0 元。
----去掉开头不完整词语----
old: 线33672885、33682885，
new: 33672885、33682885
----去掉开头不完整词语----
old: 过6个月。
new: 6个月。
old: 实行1个月房租免收、2个月租金减半优惠政策，
new: 实行1个月房租免收、2个月租金减半优惠政策
old: 成都市税务局与建设银行四川省分行联合推出小微企业“减税抗疫贷”，对防疫物资生产企业加大优惠利率信贷支持力度，
new: 成都市税务局与建设银行四川省分行联合推出小微企业“减税抗疫贷”，对防疫物资生产企业加大优惠利率信贷支持力度
----去掉开头不完整词语----
old: 予75%
new: 75%
old: 约65万个，
new: 约65万个
old: 着力稳定粮食生产，加快恢复生猪生产，积极抓好蔬菜生产，统筹抓好家禽、草食牲畜生产以及抓好动植物疫情防控等工作，做到当前生产和长远谋划两不误。（
new: 着力稳定粮食生产，加快恢复生猪生产，积极抓好蔬菜生产，统筹抓好家禽、草食牲畜生产以及抓好动植物疫情防控等工作，做到当前生产和长远谋划两不误。
----去掉开头不完整词语----
old: 站http://hunan.chinatax.gov.cn/）。
new: http://hunan.chinatax.gov.cn/）。
----截取开头标点----
old: 绍，“减税抗疫贷”是成都市税务局与中国建设银行四川省分行为贯彻落实党中央、国务院决策部署，加大支援防疫抗疫阵线企业，主动帮助受疫情影响的小微企业渡过难关而联合推出的一款小微信贷产品。作为税银合作“减税云贷”系列产品，“减税抗疫贷”以纳税企业的实缴税额及减免税额为授信依据，重点面向诚信纳税及享受税收减免，且生产销售疫情防控重点保障物资的小微企业发放。纯信

----截取开头标点----
old: 产，一方面积极做好疫情防控，一方面全力保障生产。
new: 一方面积极做好疫情防控，一方面全力保障生产。
----去掉开头不完整词语----
old: 施28个，
new: 28个
----去掉开头停用词----
old: 的《关于进一步明确疫情防控期间返京人员有关要求的通告》
new: 《关于进一步明确疫情防控期间返京人员有关要求的通告》
----去掉开头不完整词语----
old: 户3个月综合服务费约200万元，
new: 3个月综合服务费约200万元
old: 对疫情防控物资生产企业生产经营所需的水、电、气等，
new: 对疫情防控物资生产企业生产经营所需的水、电、气等
----去掉开头不完整词语----
old: 调0.5个百分点。
new: 0.5个百分点。
----去掉开头不完整词语----
old: 台《云南省建筑施工疫情防控措施》，
new: 《云南省建筑施工疫情防控措施》
old: 从13个扩增到30个，
new: 从13个扩增到30个
----去掉开头不完整词语----
old: 忌“盲目封村”“一封了之”“一堵了之”，切忌形式主义、官僚主义，防止一刀切。
new: “盲目封村”“一封了之”“一堵了之”，切忌形式主义、官僚主义，防止一刀切。
----截取开头标点----
old: 员，一律由乡镇（办事处）集中隔离观察，费用自理。
new: 一律由乡镇（办事处）集中隔离观察，费用自理。
----去掉开头不完整词语----
old: 过12366纳税服务热线、微信等渠道
new: 12366纳税服务热线、微信等渠道
----去掉开头不完整词语----
old: 取“减免缓退”等各类综合政策手段，为企业减负担降成本。初步测算，这次的政策可以为全市的企业减轻负担、降低成本300亿元以上。 　　以税收政策为例，一方面是政策上的减免。针对参与疫情防控的企业和个人，实施增值税免税和个人所得税等减免，对因疫情影响遭受重大损失的困难企业和个体工商户，减免相应的税收等。另一方面是报税上的延期。因受疫情影响，纳税人可能无法正
new: “减免缓退”等各类综合政策手段，为企业减负担降成本。初步测算，这次的政策可以为全市的企业减轻负担、降低成本300亿元以上。 　　以税收政策为例，一方面是政策上的减免。针对参与疫情防控的企业和个人

IndexError: list index out of range

In [24]:
a = 0.3
b = 0
c = 0
w1 = 1
w2 = 0
w3 = 0
w4 = 0

for b in np.arange(0, 1.01, 0.1):
    if b > 1 - a:
        continue

    if w2 > w1 or w3 > w2 or w4 > w3:
        continue

    df_pred_select = get_result(eval_preds, a, b, c, w1, w2, w3, w4)

    rst.append((a, b, c, w1, w2, w3, w4, rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']))
    print(rst[-1])
    
sorted(rst, key=lambda x:x[-1]['f'], reverse=True)

(0.3, 0.0, 0, 1, 0, 0, 0, {'f': 0.7417943226934051, 'p': 0.7639534056057711, 'r': 0.7877331767819972})
(0.3, 0.1, 0, 1, 0, 0, 0, {'f': 0.7405665603995328, 'p': 0.7639318337250748, 'r': 0.7859105192249332})
(0.3, 0.2, 0, 1, 0, 0, 0, {'f': 0.7352829684405444, 'p': 0.7583816286973822, 'r': 0.7804628383415158})
(0.3, 0.30000000000000004, 0, 1, 0, 0, 0, {'f': 0.7346378556949982, 'p': 0.7573649616838475, 'r': 0.7796699796591691})
(0.3, 0.4, 0, 1, 0, 0, 0, {'f': 0.7344581333732905, 'p': 0.7567656742176463, 'r': 0.7796198849827801})
(0.3, 0.5, 0, 1, 0, 0, 0, {'f': 0.7356949746099566, 'p': 0.7574323248332725, 'r': 0.7824144929222072})
(0.3, 0.6000000000000001, 0, 1, 0, 0, 0, {'f': 0.735948535159539, 'p': 0.757746427494352, 'r': 0.7813628245667159})


[(0.30000000000000004,
  0,
  0.0,
  1,
  0,
  0,
  0,
  {'f': 0.7417943226934051, 'p': 0.7639534056057711, 'r': 0.7877331767819972}),
 (0.3,
  0.0,
  0,
  1,
  0,
  0,
  0,
  {'f': 0.7417943226934051, 'p': 0.7639534056057711, 'r': 0.7877331767819972}),
 (0.4,
  0,
  0.0,
  1,
  0,
  0,
  0,
  {'f': 0.7412340943570221, 'p': 0.7644344526130057, 'r': 0.7867126743483648}),
 (0.3,
  0.1,
  0,
  1,
  0,
  0,
  0,
  {'f': 0.7405665603995328, 'p': 0.7639318337250748, 'r': 0.7859105192249332}),
 (0.4,
  0,
  0.1,
  1,
  0,
  0,
  0,
  {'f': 0.7400452785293465, 'p': 0.762135806042102, 'r': 0.7870757279418331}),
 (0.4,
  0,
  0.2,
  1,
  0,
  0,
  0,
  {'f': 0.7397496386440444, 'p': 0.7607212748992007, 'r': 0.7880405813900075}),
 (0.30000000000000004,
  0,
  0.30000000000000004,
  1,
  0,
  0,
  0,
  {'f': 0.7395830811019218, 'p': 0.7607774374596743, 'r': 0.7848905036211646}),
 (0.4,
  0,
  0.30000000000000004,
  1,
  0,
  0,
  0,
  {'f': 0.7394520710134704, 'p': 0.760075408357962, 'r': 0.785889

In [25]:
a = 0.3
b = 0
c = 0
w1 = 1
w2 = 0
w3 = 0
w4 = 0

for w2 in np.arange(0, 1.01, 0.1):
    for w3 in np.arange(0, 1.01, 0.1):
        if b > 1 - a:
            continue

        if w2 > w1 or w3 > w2 or w4 > w3:
            continue

        df_pred_select = get_result(eval_preds, a, b, c, w1, w2, w3, w4)

        rst.append((a, b, w1, w2, w3, w4, rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']))
        print(rst[-1])

sorted(rst, key=lambda x:x[-1]['f'], reverse=True)

(0.3, 0, 1, 0.0, 0.0, 0, {'f': 0.7417943226934051, 'p': 0.7639534056057711, 'r': 0.7877331767819972})
(0.3, 0, 1, 0.1, 0.0, 0, {'f': 0.7409703468177259, 'p': 0.7635435389987214, 'r': 0.78604613143298})
(0.3, 0, 1, 0.1, 0.1, 0, {'f': 0.7402017230977029, 'p': 0.7623691920235715, 'r': 0.787155471455801})
(0.3, 0, 1, 0.2, 0.0, 0, {'f': 0.7401044017192231, 'p': 0.7618647814637535, 'r': 0.7868928233058289})
(0.3, 0, 1, 0.2, 0.1, 0, {'f': 0.7383614709264933, 'p': 0.7602106964959502, 'r': 0.7856901578349453})
(0.3, 0, 1, 0.2, 0.2, 0, {'f': 0.7369530361550293, 'p': 0.7589200698532107, 'r': 0.7841252126080283})
(0.3, 0, 1, 0.30000000000000004, 0.0, 0, {'f': 0.7388775862618792, 'p': 0.7601594554033863, 'r': 0.7854275096849732})
(0.3, 0, 1, 0.30000000000000004, 0.1, 0, {'f': 0.7389749076403591, 'p': 0.7606638659632041, 'r': 0.7856901578349453})
(0.3, 0, 1, 0.30000000000000004, 0.2, 0, {'f': 0.7364799444415189, 'p': 0.7582168606992973, 'r': 0.7840363070818479})
(0.3, 0, 1, 0.30000000000000004, 0.30

KeyboardInterrupt: 

In [26]:
a = 0
b = 0
c = 1
w1 = 1
w2 = 0
w3 = 0
w4 = 0

for a in np.arange(0, 1.01, 0.1):
    for b in np.arange(0, 1.01, 0.1):
# for c in np.arange(0, 1.01, 0.1):
        if a + b > 1:
            continue

        if w2 > w1 or w3 > w2 or w4 > w3:
            continue

        df_pred_select = get_result(eval_preds, a, b, c, w1, w2, w3, w4)

        rst.append((a, b, c, w1, w2, w3, w4, rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']))
        print(rst[-1])
    
sorted(rst, key=lambda x:x[-1]['f'], reverse=True)

(0.0, 0.0, 1, 1, 0, 0, 0, {'f': 0.6351099875781138, 'p': 0.6616806313898261, 'r': 0.6780108189127545})
(0.0, 0.1, 1, 1, 0, 0, 0, {'f': 0.7074141720599094, 'p': 0.7309485358258536, 'r': 0.7513123659600623})
(0.0, 0.2, 1, 1, 0, 0, 0, {'f': 0.7318560526030914, 'p': 0.7546554379494445, 'r': 0.7739575953875553})
(0.0, 0.30000000000000004, 1, 1, 0, 0, 0, {'f': 0.7358924223689042, 'p': 0.7576996108694137, 'r': 0.7809776990479846})
(0.0, 0.4, 1, 1, 0, 0, 0, {'f': 0.7336708603847738, 'p': 0.7565050829143223, 'r': 0.7779739623420346})
(0.0, 0.5, 1, 1, 0, 0, 0, {'f': 0.7286733396753211, 'p': 0.7509350882757987, 'r': 0.7733009256827434})
(0.0, 0.6000000000000001, 1, 1, 0, 0, 0, {'f': 0.7295464920151588, 'p': 0.7522598945715702, 'r': 0.7739383541496488})
(0.0, 0.7000000000000001, 1, 1, 0, 0, 0, {'f': 0.7294125099044811, 'p': 0.7513139650986635, 'r': 0.7742106696486185})
(0.0, 0.8, 1, 1, 0, 0, 0, {'f': 0.729531548964606, 'p': 0.7516005643086909, 'r': 0.7739185676554007})
(0.0, 0.9, 1, 1, 0, 0, 0, {'

[(0.0,
  0.30000000000000004,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7358924223689042, 'p': 0.7576996108694137, 'r': 0.7809776990479846}),
 (0.0,
  0.4,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7336708603847738, 'p': 0.7565050829143223, 'r': 0.7779739623420346}),
 (0.1,
  0.30000000000000004,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.733187111794576, 'p': 0.7548445750049889, 'r': 0.7777117567966125}),
 (0.2,
  0.30000000000000004,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7329479389847015, 'p': 0.7563118357458293, 'r': 0.7757554215017838}),
 (0.1,
  0.2,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7329306724897303, 'p': 0.755880858864528, 'r': 0.7746299886243011}),
 (0.0,
  0.2,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7318560526030914, 'p': 0.7546554379494445, 'r': 0.7739575953875553}),
 (0.2,
  0.2,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.7311028544977403, 'p': 0.754441093641581, 'r': 0.7720399886918352}),
 (0.30000000000000004,
  0.2,
  1,
  1,
  0,
  0,
  0,
  {'f': 0.730300523821158, 'p': 0.7532431592801484, 'r': 0.77391144

In [27]:
a = 0
b = 0.3
c = 1
w1 = 1
w2 = 0
w3 = 0
w4 = 0

for c in np.arange(0, 1.01, 0.1):
#     for b in np.arange(0, 1.01, 0.1):
# for c in np.arange(0, 1.01, 0.1):
    if b > 1 - a:
        continue

    if w2 > w1 or w3 > w2 or w4 > w3:
        continue

    df_pred_select = get_result(eval_preds, a, b, c, w1, w2, w3, w4)

    rst.append((a, b, c, w1, w2, w3, w4, rouge.get_scores(df_pred_select['answer_pred_refine'].apply(spacify), df_pred_select['answer'].apply(spacify), avg=True)['rouge-l']))
    print(rst[-1])
    
sorted(rst, key=lambda x:x[-1]['f'], reverse=True)

(0, 0.3, 0.0, 1, 0, 0, 0, {'f': 0.7381819295910711, 'p': 0.7601224400646625, 'r': 0.7849168823335569})


KeyboardInterrupt: 

In [12]:
# eval_preds = pickle.load(open('tmp/val-preds-v7.p', 'rb'))

In [22]:
import sys
sys.setrecursionlimit(10000)

In [24]:
# 0.660 /models/albert-chinese-large-v5  answer_pred_refine a = 0.9
# 0.663 /models/albert-chinese-large-v5  answer_pred_refine a = 0.9 thres = 0.99
# 687 ./models/albert-chinese-large-v5.2-webqa/ a = 0.8 thres = 0.99
# 689 ./models/albert-chinese-large-v5.2-webqa/ a = 0.8 thres = 0.9
# 0.694 ./models/albert-chinese-large-v6/ a = 0.2 thres = 0.9
# 0.669 albert-chinese-large-v7/checkpoint-2500/ a = 0.8 thres = 0.8